In [2]:
import sys
sys.path.append('../utils/')
from parsing import *
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

PATH_INTERMEDIATE = '../../data/intermediate/'
PATH_RAW = '../../data/raw/'
PATH_FINAL = '../../data/final/'

## OZP

In [41]:
insurance_companies = pd.DataFrame()

In [42]:
ozp = pd.read_excel(PATH_RAW+'ic/ozp.xlsx')

In [43]:
ozp.columns = ['year', 'facility_id', 'expertise', 'count'] # rok, identifikacni cislo strediska, odbornost, pocet vykonu
ozp['insurance_company'] = 'OZP'
ozp.info()
insurance_companies = insurance_companies.append(ozp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343327 entries, 0 to 343326
Data columns (total 5 columns):
year                 343327 non-null int64
facility_id          343327 non-null int64
expertise            343325 non-null object
count                343327 non-null int64
insurance_company    343327 non-null object
dtypes: int64(3), object(2)
memory usage: 13.1+ MB


## VZP

In [44]:
vzp = pd.read_excel(PATH_RAW+'ic/vzp.xlsx')

In [45]:
vzp.columns = ['year', 'expertise', 'procedure_code', 'region', 'count'] # rok, odbornost, kod vykonu, region, pocet vykonu
vzp['insurance_company'] = 'VZP'
vzp.info()
insurance_companies = insurance_companies.append(vzp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455390 entries, 0 to 455389
Data columns (total 6 columns):
year                 455390 non-null int64
expertise            455390 non-null object
procedure_code       455390 non-null int64
region               455390 non-null object
count                455390 non-null int64
insurance_company    455390 non-null object
dtypes: int64(3), object(3)
memory usage: 20.8+ MB


## SKODA

In [46]:
skoda = pd.read_csv(PATH_RAW+'ic/skoda.csv', sep=';', usecols=['odb','sum_vykony','ico','rok'])

In [47]:
skoda.columns = ['expertise', 'count', 'facility_id', 'year'] # rok, odbornost, kod vykonu, region, pocet vykonu
skoda['insurance_company'] = 'SKODA'
skoda.info()
insurance_companies = insurance_companies.append(skoda)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80502 entries, 0 to 80501
Data columns (total 5 columns):
expertise            80491 non-null object
count                80502 non-null int64
facility_id          80435 non-null float64
year                 80502 non-null int64
insurance_company    80502 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 3.1+ MB


In [44]:
codes = pd.read_excel(PATH_RAW+'ic/codes.xlsx')

codes

,Kód,Název,Pořadí,Sazba režie
0,901,klinická psychologie,9010,3.38
1,902,fyzioterapeut,9020,3.38
2,903,klinická logopedie,9030,3.38
3,904,jiný zdravotnický pracovník,9040,3.38
4,905,zrakový terapeut,9050,3.38
...,...,...,...,...
165,403,radiační onkologie,4030,3.38
166,404,dermatovenerologie,4050,3.38
167,405,dětská dermatologie,4060,3.38
168,406,korektivní dermatologie,4070,3.38


In [70]:
insurance_companies.reset_index().to_feather(PATH_INTERMEDIATE+'insurance_companies.ftr')
insurance_companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879219 entries, 0 to 80501
Data columns (total 7 columns):
count                879219 non-null int64
expertise            879206 non-null object
facilities_id        423762 non-null float64
insurance_company    879219 non-null object
procedure_code       455390 non-null float64
region               455390 non-null object
year                 879219 non-null int64
dtypes: float64(2), int64(2), object(3)
memory usage: 53.7+ MB


## NRPZS

In [18]:
# nrpzs = pd.read_csv('https://opendata.mzcr.cz/data/nrpzs/narodni-registr-poskytovatelu-zdravotnich-sluzeb.csv') # 25. 9. 2022

nrpzs = pd.read_csv(PATH_RAW+'nrpzs.csv')

nrpzs.columns = [re.sub(r"(?<=[a-z])([A-Z])", r"_\1", x).lower() for x in nrpzs.columns]
nrpzs.to_csv(PATH_FINAL+'nrpzs.csv')

Index(['MistoPoskytovaniId', 'ZdravotnickeZarizeniId', 'Kod', 'NazevZarizeni',
       'DruhZarizeni', 'Obec', 'Psc', 'Ulice', 'CisloDomovniOrientacni',
       'Kraj', 'KrajCode', 'Okres', 'OkresCode', 'SpravniObvod',
       'PoskytovatelTelefon', 'PoskytovatelFax', 'PoskytovatelEmail',
       'PoskytovatelWeb', 'Ico', 'TypOsoby', 'PravniFormaKod', 'KrajCodeSidlo',
       'OkresCodeSidlo', 'ObecSidlo', 'PscSidlo', 'UliceSidlo',
       'CisloDomovniOrientacniSidlo', 'OborPece', 'FormaPece', 'DruhPece',
       'OdbornyZastupce', 'Lat', 'Lng'],
      dtype='object')

In [30]:
ic = pd.read_feather(PATH_INTERMEDIATE+'insurance_companies.ftr').set_index('index')
ic.expertise.value_counts()

014    67456
001    57960
002    27436
603    22466
5H1    21098
       ...  
016        1
5I4        1
2I5        1
2F6        1
000        1
Name: expertise, Length: 330, dtype: int64

In [76]:
with open(PATH_RAW+'ic/ODBORN', 'r', encoding="utf-8") as f:
    codes = pd.DataFrame([x.split(';') for x in f.read().splitlines()], columns=['expertise_code', 'expertise_name'])

In [92]:
set(codes.expertise_code.unique()).difference(set(ic.expertise.unique()))
ax = set(ic.expertise.unique()).difference(set(codes.expertise_code.unique()))
ic[ic.expertise == 'XNA']

,count,expertise,facilities_id,insurance_company,procedure_code,region,year
index,,,,,,,
192664,24005779,XNA,NaN,VZP,9552.0,Čechy,2015
192665,6685505,XNA,NaN,VZP,9552.0,Morava,2015
192666,6998477,XNA,NaN,VZP,9552.0,Slezsko,2015
233849,24249586,XNA,NaN,VZP,9552.0,Čechy,2016
233850,6716164,XNA,NaN,VZP,9552.0,Morava,2016
233851,6930549,XNA,NaN,VZP,9552.0,Slezsko,2016
275071,24499864,XNA,NaN,VZP,9552.0,Čechy,2017
275072,6713107,XNA,NaN,VZP,9552.0,Morava,2017
275073,6923110,XNA,NaN,VZP,9552.0,Slezsko,2017


In [25]:
# TODO namergovat ica pro ozp a skodu
# pro vzp vzit kraje a nejak to rozsekat a priradit cechy, moravu a slezsko``

In [26]:
f_ids = set(ic.facilities_id.unique())
icos = set(nrpzs.Ico.unique())
len(f_ids.difference(icos)), len(icos.difference(f_ids))

pd.merge(ic, nrpzs, left_on='facilities_id', right_on='Ico')

,count,expertise,facilities_id,insurance_company,procedure_code,region,year,MistoPoskytovaniId,ZdravotnickeZarizeniId,NazevZarizeni,...,Ulice,CisloDomovniOrientacni,Kraj,Okres,SpravniObvod,OborPece,FormaPece,DruhPece,Lat,Lng
0,13,002,844641.0,OZP,NaN,None,2017,239894,161417,"Sdružené zdravotnické zařízení Krnov, příspěvk...",...,Tyršova,596,Moravskoslezský kraj,Bruntál,NaN,zubní lékařství,primární ambulantní péče,NaN,49.967079,17.604306
1,13,002,844641.0,OZP,NaN,None,2017,238943,160596,"Sdružené zdravotnické zařízení Krnov, příspěvk...",...,Smetanova,1099/20a,Moravskoslezský kraj,Bruntál,NaN,pneumologie a ftizeologie,specializovaná ambulantní péče,NaN,49.992122,17.473078
2,13,002,844641.0,OZP,NaN,None,2017,238809,160474,"Sdružené zdravotnické zařízení Krnov, příspěvk...",...,Jesenická,448/9,Moravskoslezský kraj,Bruntál,NaN,zubní lékařství,primární ambulantní péče,NaN,50.121876,17.375167
3,13,002,844641.0,OZP,NaN,None,2017,227685,139458,"Sdružené zdravotnické zařízení Krnov, příspěvk...",...,Komenského,318,Moravskoslezský kraj,Bruntál,NaN,zubní lékařství,primární ambulantní péče,NaN,49.915337,17.377391
4,13,002,844641.0,OZP,NaN,None,2017,157790,53387,"Sdružené zdravotnické zařízení Krnov, p.o. MSK...",...,Nemocniční,287,Moravskoslezský kraj,Bruntál,NaN,NaN,následná lůžková péče - standardní,ošetřovatelská péče,49.827969,17.545516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080965,134,404,8919411.0,SKODA,NaN,None,2021,227762,139527,Dermatologie prof. Hercogové s.r.o.,...,Dukelských hrdinů,848/17,Hlavní město Praha,Hlavní město Praha,NaN,dermatovenerologie,ambulantní péče,NaN,50.097496,14.432845
1080966,134,404,8919411.0,SKODA,NaN,None,2021,226539,138680,Dermatologie prof. Hercogové s.r.o.,...,Tychonova,44/3,Hlavní město Praha,Hlavní město Praha,NaN,dermatovenerologie,ambulantní péče,NaN,50.096116,14.403063
1080967,3,958,73255386.0,SKODA,NaN,None,2021,239147,160775,Jaroslava Horáčková Kováříková,...,Míru,355,Olomoucký kraj,Jeseník,NaN,Všeobecná sestra,ambulantní péče,NaN,50.389954,16.998133
1080968,7,958,2227126.0,SKODA,NaN,None,2021,193825,106986,"Ústav vývoje a klinických aplikací, z.ú.",...,Kochova,815/1,Moravskoslezský kraj,Karviná,NaN,Všeobecná sestra,zdrav. péče poskytovaná ve vlastním soc. prost...,ošetřovatelská péče,49.795873,18.406717


,Kód,Název,Pořadí,Sazba režie
0,901,klinická psychologie,9010,3.38
1,902,fyzioterapeut,9020,3.38
2,903,klinická logopedie,9030,3.38
3,904,jiný zdravotnický pracovník,9040,3.38
4,905,zrakový terapeut,9050,3.38
...,...,...,...,...
165,403,radiační onkologie,4030,3.38
166,404,dermatovenerologie,4050,3.38
167,405,dětská dermatologie,4060,3.38
168,406,korektivní dermatologie,4070,3.38
